**Indian Liver Patient Keras Sequential model.**

In [0]:
import tensorflow as tf
from tensorflow.keras import Sequential,layers
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.preprocessing import MinMaxScaler,Normalizer

Data Preparation using Pandas and Sklearn libraries.

In [0]:
class Preprocess(object):
  def __init__(self):
    self.url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00225/Indian%20Liver%20Patient%20Dataset%20(ILPD).csv"
    self.cols = ['age','gender','TB','DB','alkphos','sgpt','sgot','TP','albumin','AG_ratio','Is_liverPatient']
    self.dataset = None
  
  def load_data(self):
    self.dataset = pd.read_csv(self.url,names=self.cols)
    print('shape of df :',self.dataset.shape)
    return self.dataset
  
  def preprocess(self,data):
    #preprocessing the data
    # 1. Removing the duplicate rows
    self.dataset = data.drop_duplicates()
    print('Shape after removing the duplicates :',self.dataset.shape)
    # checking for the Null values, #display no of null values by column
    print(self.dataset.isnull().sum())
    #dropping the 4 rows here
    self.dataset = self.dataset[pd.notnull(self.dataset['AG_ratio'])]
    return None
  
  def encode_label(self):
    #changing the gender attribute to categorical type
    # 0 is for 'Female' and 1 for 'Male
    self.dataset.gender = pd.factorize(self.dataset.gender)[0] + 0.0
    #Encoding the label for proper implementation in network (1 neuron in the output layer)
    # Label 1 means "a liver patient" so encoding as 1
    # Label 2 means "not a liver patient" so encoding as 0
    self.dataset.loc[(self.dataset.Is_liverPatient == 2), 'Is_liverPatient'] = 0
    return None
  
  def scaleData(self):
    #Scaling the attribute values to be between [0,1]
    scaler = MinMaxScaler()
    cols_to_scale = ['age', 'TB', 'DB', 'alkphos', 'sgpt', 'sgot', 'TP', 'albumin', 'AG_ratio']
    Dataset = pd.DataFrame(data=self.dataset)
    self.dataset[cols_to_scale] = scaler.fit_transform(Dataset[cols_to_scale])
    return None
  
  def get_dataset(self):
    return self.dataset

In [0]:
prep = Preprocess()
dataset = prep.load_data()
prep.preprocess(dataset)
prep.encode_label()
prep.scaleData()
dataset = prep.get_dataset()

shape of df : (583, 11)
Shape after removing the duplicates : (570, 11)
age                0
gender             0
TB                 0
DB                 0
alkphos            0
sgpt               0
sgot               0
TP                 0
albumin            0
AG_ratio           4
Is_liverPatient    0
dtype: int64


In [0]:
#Splitting the dataset into train and test splits
# Taking the label out of the original preprocessed dataset
X = dataset[['age', 'gender', 'TB', 'DB', 'alkphos', 'sgpt', 'sgot', 'TP', 'albumin', 'AG_ratio']]
y = dataset['Is_liverPatient']

X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.70)  #test_size = 1-0.7 = 0.3
#Changing them into numpy array as the last step
X_train = X_train.to_numpy().astype(np.float32)
y_train = y_train.to_numpy().astype(np.int32)
X_test = X_test.to_numpy().astype(np.float32)
y_test = y_test.to_numpy().astype(np.int32)
print('X_train shape :',X_train.shape)
print('y_train shape :',y_train.shape)
print('X_test shape :',X_test.shape)
print('y_test shape :',y_test.shape)

X_train shape : (396, 10)
y_train shape : (396,)
X_test shape : (170, 10)
y_test shape : (170,)


Creating the keras Sequential Model and training it.

In [0]:
#main procedure call

#creating the tensors out of the numpy arrays
'''
X_train = tf.Variable(X_train, dtype=tf.float32, trainable=False)
y_train = tf.Variable(y_train, dtype=tf.int32, trainable=False)
X_test = tf.Variable(X_test, dtype=tf.float32, trainable=False)
y_test = tf.Variable(y_test, dtype=tf.int32, trainable=False)
'''
#intializing the network parameters
input_dimension = X_train.shape[1]
num_classes = 1
#layer_activations = ["Relu", "Relu", "Softmax"]
#layer_activations = ["Relu", "Softmax"]
#nodes_in_layers = [5, num_classes]

#intializing the hyperparameters
batch_size = 50
epochs = 10
alpha = 0.1

#creating the tensor datasets for incorporating the batches
#train_ds = tf.data.Dataset.from_tensor_slices((X_train,y_train)).shuffle(100).batch(batch_size)
#test_ds = tf.data.Dataset.from_tensor_slices((X_test,y_test)).batch(batch_size)


In [0]:
model = Sequential()
model.add(layers.Dense(units=10,activation='relu'))
model.add(layers.Dense(units=1,activation='softmax'))

In [0]:
model.build(input_shape=(1,10))
model.call(tf.keras.Input(shape=(1,10)))
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 1, 10)             110       
_________________________________________________________________
dense_10 (Dense)             (None, 1, 1)              11        
Total params: 121
Trainable params: 121
Non-trainable params: 0
_________________________________________________________________


In [0]:
for layer in model.layers:
  print('weight shape :',layer.get_weights()[0].shape)
  print('bias shape :',layer.get_weights()[1].shape)

weight shape : (10, 5)
bias shape : (5,)
weight shape : (5, 1)
bias shape : (1,)


In [0]:
#model compiled with sgd optimizer
model.compile(optimizer='sgd',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [0]:
#model compiled with adam optimizer
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [0]:
history = model.fit(X_train,y_train,epochs=epochs,validation_data=(X_test,y_test))

Epoch 1/10
13/13 [==============================] - 0s 9ms/step - loss: 0.6112 - accuracy: 0.7020 - val_loss: 0.5721 - val_accuracy: 0.7412
Epoch 2/10
13/13 [==============================] - 0s 4ms/step - loss: 0.6112 - accuracy: 0.7020 - val_loss: 0.5721 - val_accuracy: 0.7412
Epoch 3/10
13/13 [==============================] - 0s 4ms/step - loss: 0.6112 - accuracy: 0.7020 - val_loss: 0.5721 - val_accuracy: 0.7412
Epoch 4/10
13/13 [==============================] - 0s 4ms/step - loss: 0.6112 - accuracy: 0.7020 - val_loss: 0.5721 - val_accuracy: 0.7412
Epoch 5/10
13/13 [==============================] - 0s 4ms/step - loss: 0.6112 - accuracy: 0.7020 - val_loss: 0.5721 - val_accuracy: 0.7412
Epoch 6/10
13/13 [==============================] - 0s 4ms/step - loss: 0.6112 - accuracy: 0.7020 - val_loss: 0.5721 - val_accuracy: 0.7412
Epoch 7/10
13/13 [==============================] - 0s 4ms/step - loss: 0.6112 - accuracy: 0.7020 - val_loss: 0.5721 - val_accuracy: 0.7412
Epoch 8/10
13/13 [==

In [0]:
#evaluating the model
test_loss,test_acc = model.evaluate(X_test,y_test,verbose=2)

6/6 - 0s - loss: 0.5721 - accuracy: 0.7412


**Results:-**

a) ***with SGD optimizer***

The train loss = 0.6112, train acccuracy = 70.20%, val_loss = 0.5721 and val_accuracy = 74.12%

b) ***with adam optimizer***

